In [1]:
import sys
sys.path.insert(0, '../src/')

In [2]:
import torch
from torch import nn
import numpy as np
from preconditioners.datasets import CenteredGaussianDataset
from preconditioners.optimizers import PrecondGD

In [3]:
dataset = CenteredGaussianDataset(w_star=np.ones(2), d = 2, c=np.ones((2, 2)), n=1000)

/home/wilcoln/Workspace/Projects/preconditioners/notebooks/../src/preconditioners/utils.py:219: UserWarning: Warning, norms of datapoints are not sqrt(d)
  warnings.warn('Warning, norms of datapoints are not sqrt(d)')


In [4]:
class MLP(nn.Module):
    """
      Multilayer Perceptron for regression.
    """

    def __init__(self, in_size):
        super().__init__()
        self.layers = nn.Sequential(
#             nn.Linear(in_size, 1),
            nn.Linear(in_size, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        """
          Forward pass
        """
        return self.layers(x)

In [5]:
NB_EPOCHS = 2
# Set fixed random number seed
torch.manual_seed(0)
# Split dataset
train_size = int(0.7 * len(dataset))
test_size = int(0.2 * len(dataset))
extra_size = len(dataset) - train_size - test_size

train_dataset, test_dataset, extra_dataset = torch.utils.data.random_split(dataset, [train_size, test_size, extra_size])

# Create trainloader object
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=1)

# Create testloader object
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True, num_workers=1)

# Initialize the MLP
mlp = MLP(in_size=dataset.X.shape[1])

# Define the loss function and optimizer
loss_function = nn.MSELoss()
# optimizer = torch.optim.Adam(mlp.parameters(), lr=1e-4)

# num_params = sum(p.numel() for p in mlp.parameters())
# p_inv = np.zeros((num_params, num_params))
# p_inv = np.eye(num_params)
optimizer = PrecondGD(mlp, lr=1e-4, labeled_data=train_dataset[:][0], unlabeled_data=extra_dataset[:][0])


losses = {'train': [], 'test': []}
epochs = range(NB_EPOCHS)


def train(epoch):
     # Print epoch
#     print(f'Starting epoch {epoch + 1}')

    mlp.train()
    
    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data
    for i, data in enumerate(trainloader, 0):

        # Get and prepare inputs
        inputs, targets = data
        inputs, targets = inputs.float(), targets.float()
        targets = targets.reshape((targets.shape[0], 1))

        # Zero the gradients
        optimizer.zero_grad()

        # Perform forward pass
        outputs = mlp(inputs)

        # Compute loss
        loss = loss_function(outputs, targets)

        # Perform backward pass
        loss.backward()

        # Perform optimization
        optimizer.step()

        # Print statistics
        current_loss += loss.item()
    
    current_loss /= len(trainloader)
    
    losses['train'].append(current_loss)
    
#     print('Train Loss: %.3f'%(current_loss))

    
def test(epoch):
    mlp.eval()

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data
    for i, data in enumerate(testloader, 0):

        # Get and prepare inputs
        inputs, targets = data
        inputs, targets = inputs.float(), targets.float()
        targets = targets.reshape((targets.shape[0], 1))

        # Perform forward pass
        outputs = mlp(inputs)

        # Compute loss
        loss = loss_function(outputs, targets)

        # Print statistics
        current_loss += loss.item()
    
    current_loss /= len(testloader)
    
    losses['test'].append(current_loss)
    
#     print('Test Loss: %.3f'%(current_loss))

MLP(
  (layers): Sequential(
    (0): Linear(in_features=2, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=1, bias=True)
  )
)
=> We keep the following layers in PrecondGD. 
(0): Linear(in_features=2, out_features=64, bias=True)
(1): Linear(in_features=64, out_features=32, bias=True)
(2): Linear(in_features=32, out_features=1, bias=True)


In [6]:
# Run the training loop
for epoch in epochs:
    train(epoch)
    test(epoch)

RuntimeError: One of the differentiated Tensors does not require grad

In [ ]:
import matplotlib.pyplot as plt

plt.plot(losses['train'], '-o')
plt.plot(losses['test'], '-o')
plt.xlabel('Epochs')
plt.ylabel('Losses')
plt.legend(['Train','Valid'])
plt.title('Train vs Valid Losses')
 
plt.show()

In [ ]:
from torch.autograd import grad 

x = torch.ones(2, 2, requires_grad=True)

y = (x+1) ** 2

print(x)
print(y)
dy_dx = grad(outputs=y, inputs=x, grad_outputs=torch.ones_like(y))
print(f'dy/dx:\n {dy_dx}')